In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
import os
import time
import matplotlib.pyplot as plt

# head (머리)

In [2]:
actions = ['head']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [7]:
cap = cv2.VideoCapture('./data_video/head/head_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

head (972, 141)
head (942, 30, 141)


# shoulder (어깨)

In [8]:
actions = ['shoulder']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [13]:
cap = cv2.VideoCapture('./data_video/shoulder/shoulder_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 1)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

shoulder (1054, 141)
shoulder (1024, 30, 141)


# knee (무릎)

In [14]:
actions = ['knee']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [19]:
cap = cv2.VideoCapture('./data_video/knee/knee_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 2)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

knee (952, 141)
knee (922, 30, 141)


# clap (박수 짝)

In [20]:
actions = ['clap']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [25]:
cap = cv2.VideoCapture('./data_video/clap/clap_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 3)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

clap (1081, 141)
clap (1051, 30, 141)


# left_foot (왼발 쿵)

In [26]:
actions = ['left_foot']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [31]:
cap = cv2.VideoCapture('./data_video/left_foot/left_foot_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 4)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

left_foot (1021, 141)
left_foot (991, 30, 141)


# right_foot (오른발 쿵)

In [32]:
actions = ['right_foot']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [38]:
cap = cv2.VideoCapture('./data_video/right_foot/right_foot_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 5)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

right_foot (914, 141)
right_foot (884, 30, 141)


# both_foot (양발 쿵)

In [39]:
actions = ['both_foot']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [44]:
cap = cv2.VideoCapture('./data_video/both_foot/both_foot_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 6)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

both_foot (750, 141)
both_foot (720, 30, 141)


#  yeah (예!)

In [45]:
actions = ['yeah']
seq_length = 30
secs_for_action = 30

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,)

In [51]:
cap = cv2.VideoCapture('./data_video/yeah/yeah_yj.mp4')
# cap = cv2.VideoCapture("http://192.168.0.9:4747/video")

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

for idx, action in enumerate(actions):
    data = []
    while cap.isOpened():
    
        ret, img = cap.read()
        
        if not ret:
            break
            
        img = cv2.resize(img, (0,0), fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks is not None:
            joint = np.zeros((33, 4))
            for j, res in enumerate(result.pose_landmarks.landmark):
                joint[j] = [res.x, res.y, res.z, res.visibility]

            # Compute angles between joints
            v1 = joint[[13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30], :3] # Parent joint
            v2 = joint[[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28], :3] # Child joint
            v = v2 - v1 # [12, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            v = np.nan_to_num(v)

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0, 1, 2, 3, 6, 7, 8, 9],:], 
                v[[2, 3, 4, 5, 8, 9, 10, 11],:])) # [8,]

            angle = np.degrees(angle) # Convert radian to degree

            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx + 7)

            d = np.concatenate([joint.flatten(), angle_label])

            data.append(d)

            mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    print(action, data.shape)
    np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    
cap.release()
cv2.destroyAllWindows()

yeah (921, 141)
yeah (891, 30, 141)
